* **참고사항: 심준교님 코드와 비교하여 epoch가 진행하고 있는데, acc가 지지부진한 것을 볼 수 있다.**

    : 정체된 acc로 2시간 가량 돌리다가 멈춤.

In [1]:
from google.colab import drive
drive.mount('/content/drive')\

Mounted at /content/drive


In [2]:
from google.colab import output
!cp -r "/content/drive/MyDrive/21-1/data" "data"

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image
from typing import Tuple, Sequence, Callable
import glob
from sklearn.model_selection import KFold
import time
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from google.colab import output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

In [5]:
os.chdir('/content/data')

# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist_2nd/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist_2nd/"
# 출력 결과 지우기
output.clear()

In [6]:
os.chdir('/content/data')
labels_df = pd.read_csv('dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('/content/data/dirty_mnist_2nd/*')))[:]
labels = np.array(labels_df.values[:,1:])
test_imgs_dir = np.array(sorted(glob.glob('/content/data/test_dirty_mnist_2nd/*')))

In [7]:
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs_dir)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass

In [8]:
resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)
#resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext101_32x8d', pretrained=True)

class Resnext(nn.Module):
    def __init__(self):
        super(Resnext, self).__init__()
        self.resnext = resnext 
        self.FC = nn.Linear(1000, 26)
        nn.init.xavier_normal_(self.FC.weight)
      
        

    def forward(self, x):
        x = self.resnext(x)
        x = torch.sigmoid(self.FC(x))
        return x

model = Resnext()

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
folds=[]
for train_idx, valid_idx in kf.split(labels_df):
    folds.append((train_idx, valid_idx))

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True

In [10]:
seed_everything(42)
print('start')
for fold in range(1):
    model = Resnext().to(device)
#     model = nn.DataParallel(model)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]



    train_transform = transforms.Compose([                                
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
        ])
    valid_transform = transforms.Compose([                                 
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
        ])


    epochs=30
    batch_size=64         # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.
    
    
    
    # Data Loader
    train_dataset = MnistDataset_v1(imgs_dir=imgs_dir[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

    valid_dataset = MnistDataset_v1(imgs_dir=imgs_dir[valid_idx], labels=labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)  
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    # 
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.003, weight_decay=1e-7)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones= [23,29], gamma=0.1)

    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0

    best_models=[]

    for epoch in range(epochs):
      with tqdm(train_loader,total=train_loader.__len__(),unit='batch',position = 0) as train_bar:
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_bar)):
            train_bar.set_description(f"Train Epoch {epoch}")
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()


            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())
            train_acc = np.mean(batch_accuracy_list)
            
            train_bar.set_postfix(train_loss= loss.item(),train_acc = train_acc)

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []

        with torch.no_grad():
          with tqdm(valid_loader,total=valid_loader.__len__(),unit="batch",position = 0) as valid_bar:
            for n_valid, (X_valid, y_valid) in enumerate((valid_bar)):
                valid_bar.set_description(f"Valid Epoch {epoch}")
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                val_acc=np.mean(valid_batch_accuracy)
                valid_bar.set_postfix(valid_loss = valid_loss,valid_acc = val_acc)
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))

        scheduler.step()

        if np.mean(valid_batch_accuracy) > 0.70:
            path = "/content/drive/MyDrive/21-1/models_resnext/"
            MODEL = "resnext101"
            torch.save(model, f'{path}_{MODEL}_{valid_loss:2.4f}_epoch_{epoch}.pth')
            print('saved model')
            path = "/content/drive/MyDrive/21-1/models_resnext_state/"
            MODEL = "resnext101"
            torch.save(model.state_dict(), f'{path}_{MODEL}_{valid_loss:2.4f}_epoch_{epoch}.pth')
            print('saved model path')

        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            best_model=model
            valid_best_accuracy = np.mean(valid_batch_accuracy)

    best_models.append(best_model)

start


Train Epoch 0:   0%|          | 0/625 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Valid Epoch 0:   0%|          | 0/313 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: UserWarning: To copy construct from a 

KeyboardInterrupt: ignored